In [ ]:
import nglview as ng
import pandas as pd
import mdtraj as md
import os

In [ ]:
!pwd

In [ ]:
os.chdir("../")
%ls

In [ ]:
!tree

In [ ]:
os.chdir("./output_files")
%pwd

# Obtaining the input for a simulation

In [ ]:
%ls ../input_files/user_file/

In [ ]:

view = ng.show_structure_file("../input_files/user_file/2b42_Swissmodel.pdb")
view

## Cleaning the input structure

In [ ]:

!grep -v HETATM ../input_files/user_file/2b42_Swissmodel.pdb > 2b42_Swissmodel_protein_tmp.pdb
!grep -v CONECT 2b42_Swissmodel_protein_tmp.pdb > 2b42_Swissmodel_protein.pdb

The cleaned structure now looks like this:

In [ ]:
view = ng.show_structure_file("2b42_Swissmodel_protein.pdb")
view

In [ ]:
!grep MISSING ../input_files/user_file/2b42_Swissmodel.pdb

## Generating a topology

In [ ]:
!gmx pdb2gmx -f 2b42_Swissmodel_protein.pdb -o 2b42_Swissmodel_processed.gro -water tip3p -ff "charmm27"

## A peek at the generated files

In [ ]:
!ls

# Solvating the simulation system

## Defining the simulation box

In [ ]:
!gmx editconf -f 2b42_Swissmodel_processed.gro -o 2b42_Swissmodel_newbox.gro -c -d 1.0 -bt dodecahedron

In [ ]:
!gmx solvate -cp 2b42_Swissmodel_newbox.gro -cs spc216.gro -o 2b42_Swissmodel_solv.gro -p topol.top

In [ ]:
!tail topol.top

In [ ]:
view = ng.show_structure_file("2b42_Swissmodel_solv.gro")
view.add_representation(repr_type='ball+stick', selection='SOL')
view.camera='orthographic'
view

# Adding Ions

## Preparing the input for "gmx genion"

In [ ]:
!touch ions.mdp

In [ ]:
!gmx grompp -f ions.mdp -c 2b42_Swissmodel_solv.gro -p topol.top -o ions.tpr

In [ ]:
!printf "SOL\n" | gmx genion -s ions.tpr -o 2b42_Swissmodel_solv_ions.gro -conc 0.15 -p \
topol.top -pname NA -nname CL -neutral

In [ ]:
!tail -6 topol.top

In [ ]:
view = ng.show_structure_file("2b42_Swissmodel_solv_ions.gro")
view.add_representation(repr_type='spacefill', selection='NA')
view.add_representation(repr_type='spacefill', selection='CL')
view.add_representation(repr_type='ball+stick', selection='SOL')
view.camera='orthographic'
view

# Energy minimisation

In [ ]:
!cat ../input_files/file_assets/emin-charmm.mdp

In [ ]:
!gmx grompp -f ../input_files/file_assets/emin-charmm.mdp -c 2b42_Swissmodel_solv_ions.gro -p topol.top -o em.tpr

In [ ]:
!gmx mdrun -v -deffnm em -ntmpi 1 -ntomp 1

## Determining if the run was successful

## Analysing the run results

In [ ]:
!printf "Potential\n0\n" | gmx energy -f em.edr -o potential.xvg -xvg none

In [ ]:
df = pd.read_csv('potential.xvg', sep='\\s+', header=None, names=['step','energy'])
df.plot('step')

# Position restraints 

In [ ]:
!head -5 ../input_files/file_assets/nvt-charmm.mdp 

# Equilibration run - temperature

In [ ]:
!cat ../input_files/file_assets/nvt-charmm.mdp

In [ ]:
!gmx grompp -f ../input_files/file_assets/nvt-charmm.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr 
!gmx mdrun -ntmpi 1 -ntomp 8 -v -deffnm nvt

In [ ]:
!echo "Temperature" | gmx energy -f nvt.edr -o temperature.xvg -xvg none -b 20

In [ ]:
df = pd.read_csv('temperature.xvg', sep='\\s+', header=None, names=['time','temperature'])
df.plot('time')

# Equilibration run - pressure

In [ ]:
!cat ../input_files/file_assets/npt-charmm.mdp

In [ ]:
!gmx grompp -f ../input_files/file_assets/npt-charmm.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
!gmx mdrun -ntmpi 1 -ntomp 8 -v -deffnm npt

In [ ]:
!echo "Pressure" | gmx energy -f npt.edr -o pressure.xvg -xvg none

In [ ]:
df = pd.read_csv('pressure.xvg', sep='\\s+', header=None, names=['time','pressure'])
df.plot('time')

In [ ]:
!echo "Density" | gmx energy -f npt.edr -o density.xvg -xvg none

In [ ]:
df = pd.read_csv('density.xvg', sep='\\s+', header=None, names=['time','density'])
df.plot('time')

# The "production" run

In [ ]:
!cat ../input_files/file_assets/md-charmm.mdp

In [ ]:
!gmx grompp -f ../input_files/file_assets/md-charmm.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr

In [ ]:
!gmx mdrun -ntmpi 1 -ntomp 8 -v -deffnm md

# Analysis

In [ ]:
!gmx trjconv -h

In [ ]:
!printf "1\n1\n" | gmx trjconv -s md.tpr -f md.xtc -o md_center.xtc -center -pbc whole

In [ ]:
traj = md.load("md_center.xtc", top="2b42_Swissmodel_newbox.gro")
view = ng.show_mdtraj(traj)
view

## Check the minimum image convention

In [ ]:
!printf "1\n" | gmx mindist -s md.tpr -f md_center.xtc -pi -od mindist.xvg 

## Evaluate structural stability with RMSD

In [ ]:
!printf "4\n1\n" | gmx rms -s em.tpr -f md_center.xtc -o rmsd_xray.xvg -tu ns -xvg none

In [ ]:
df = pd.read_csv('rmsd_xray.xvg', sep='\\s+', header=None, names=['time','RMSD'])
df.plot('time')

## Measure compactness with radius of gyration

In [ ]:
!echo "1" | gmx gyrate -f md_center.xtc -s md.tpr -o gyrate.xvg -xvg none

In [ ]:
df = pd.read_csv('gyrate.xvg', sep='\\s+', header=None, names=['time','Rg'], usecols=[0, 1])
df.plot('time')

## Index files for more specific atom selection

In [ ]:
!printf "h\nq\n" | gmx make_ndx -f nvt.tpr -o

In [ ]:
!printf "splitch 1\nq\n" | gmx make_ndx -f nvt.tpr -o chains_make_ndx.ndx

In [ ]:
!printf "group "Protein" and mol 1\ngroup "Protein" and mol 2" | gmx select -s nvt.tpr -on chains_select.ndx

In [ ]:
!printf "17\n18\n"| gmx hbond -f md.xtc -s md.tpr  -n chains_make_ndx.ndx -num hbnum_ndx.xvg -xvg none

In [ ]:
!printf "0\n1\n"| gmx hbond -f md.xtc -s md.tpr -n chains_select.ndx -num hbnum.xvg -xvg none

In [ ]:
df = pd.read_csv('hbnum_ndx.xvg', sep='\\s+', header=None, names=['time','H-bonds'], usecols=[0, 1])
df.plot('time')
df = pd.read_csv('hbnum.xvg', sep='\\s+', header=None, names=['time','H-bonds'], usecols=[0, 1])
df.plot('time')
